## Tools

Tools are useful whenever you want a model to interact with external systems.

External systems (e.g., APIs) often require a particular input schema or payload, rather than natural language. 

When we bind an API, for example, as a tool we given the model awareness of the required input schema.

The model will choose to call a tool based upon the natural language input from the user. 

And, it will return an output that adheres to the tool's schema. 

[Many LLM providers support tool calling](https://python.langchain.com/v0.1/docs/integrations/chat/) and [tool calling interface](https://blog.langchain.dev/improving-core-tool-interfaces-and-docs-in-langchain/) in LangChain is simple. 
 
You can simply pass any Python `function` into `ChatModel.bind_tools(function)`.

![Screenshot 2024-08-19 at 7.46.28 PM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbab08dc1c17a7a57f9960_chain2.png)


In [1]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# _set_env("OPENAI_API_KEY")
_set_env("GROQ_API_KEY")

In [2]:
from  langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage
from typing_extensions import Annotated, TypedDict
from langchain_core.pydantic_v1 import BaseModel, Field

<h1> function call with out langgraph</h1>

In [16]:
class add(TypedDict):
    """Add two integers."""

    # Annotations must have the type and can optionally include a default value and description (in that order).
    a: Annotated[int, ..., "First integer"]
    b: Annotated[int, ..., "Second integer"]


class multiply(BaseModel):
    """Multiply two integers."""

    a: Annotated[int, ..., "First integer"]
    b: Annotated[int, ..., "Second integer"]


tools = [add, multiply]
llm = ChatGroq(model="llama3-8b-8192",temperature=0)
# llm = ChatGroq(model="gemma-7b-it",temperature=0)

llm_with_tools = llm.bind_tools(tools)
print(llm_with_tools)

bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x71dc9fbebf20>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x71dc9fbf42f0>, model_name='llama3-8b-8192', temperature=1e-08, groq_api_key=SecretStr('**********')) kwargs={'tools': [{'type': 'function', 'function': {'name': 'add', 'description': 'Add two integers.', 'parameters': {'type': 'object', 'properties': {'a': {'description': 'First integer', 'type': 'integer'}, 'b': {'description': 'Second integer', 'type': 'integer'}}, 'required': ['a', 'b']}}}, {'type': 'function', 'function': {'name': 'multiply', 'description': 'Multiply two integers.', 'parameters': {'type': 'object', 'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b']}}}]}


In [15]:

response = llm_with_tools.invoke("What is 3 * 12?")
print(response.additional_kwargs)
print("----------")
response = llm_with_tools.invoke("What is multiplication 3 and 12?")
print(response.additional_kwargs)

print("----------")
response = llm_with_tools.invoke("addition of 3 and 12?")
print(response.additional_kwargs)

print("----------")
response = llm_with_tools.invoke("3*12 + 4")
print(response.additional_kwargs)

{'tool_calls': [{'id': 'call_21dv', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'multiply'}, 'type': 'function'}]}
----------
{'tool_calls': [{'id': 'call_fs30', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'multiply'}, 'type': 'function'}]}
----------
{'tool_calls': [{'id': 'call_sq7m', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'add'}, 'type': 'function'}]}
----------
{'tool_calls': [{'id': 'call_h8j4', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_70k0', 'function': {'arguments': '{"a":36,"b":4}', 'name': 'add'}, 'type': 'function'}]}


In [20]:
tools = [
    {
      "type": "function",
      "function": {
        "name": "add",
        "description": "Add two integers.",
        "parameters": {
          "type": "object",
          "properties": {
            "a": {
              "description": "First integer",
              "type": "integer"
            },
            "b": {
              "description": "Second integer",
              "type": "integer"
            }
          },
          "required": [
            "a",
            "b"
          ]
        }
      }
    },
    {
      "type": "function",
      "function": {
        "name": "multiply",
        "description": "Multiply two integers.",
        "parameters": {
          "type": "object",
          "properties": {
            "a": {
              "type": "integer"
            },
            "b": {
              "type": "integer"
            }
          },
          "required": [
            "a",
            "b"
          ]
        }
      }
    }
  ]
llm = ChatGroq(model="llama3-8b-8192",temperature=0).bind(tools = tools)

response = llm.invoke("What is 3 * 12?")
print(response.additional_kwargs)
print("----------")

response = llm.invoke("What is multiplication 3 and 12?")
print(response.additional_kwargs)

print("----------")
response = llm.invoke("addition of 3 and 12?")
print(response.additional_kwargs)

print("----------")
response = llm.invoke("3*12 + 4")
print(response.additional_kwargs)


{'tool_calls': [{'id': 'call_rpda', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'multiply'}, 'type': 'function'}]}
----------
{'tool_calls': [{'id': 'call_7bn4', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'multiply'}, 'type': 'function'}]}
----------
{'tool_calls': [{'id': 'call_18my', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'add'}, 'type': 'function'}]}
----------
{'tool_calls': [{'id': 'call_kjvc', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_8bmn', 'function': {'arguments': '{"a":36,"b":4}', 'name': 'add'}, 'type': 'function'}]}


<h1> function call with langgraph</h1>

In [22]:
from langgraph.graph import MessagesState
from langgraph.graph import StateGraph,START,END

## Reducers allow us to specify how state updates are performed.
## MessagesState - inbuild reducer which appends the message
 
class MessagesState(MessagesState):
    ## Every time the message is appended
    # Add any keys needed beyond messages, which is pre-built 
    pass

Since having a list of messages in graph state is so common, LangGraph has a pre-built [`MessagesState`](https://langchain-ai.github.io/langgraph/concepts/low_level/#messagesstate)! 

`MessagesState` is defined: 

* With a pre-build single `messages` key
* This is a list of `AnyMessage` objects 
* It uses the `add_messages` reducer


In [23]:
def tool_calling_llm(state:MessagesState):
    tools = [add, multiply]
    llm = ChatGroq(model="llama3-8b-8192",temperature=0)
    llm_with_tools = llm.bind_tools(tools)
    return {"messages":llm_with_tools.invoke(state["messages"])}

graph_builder = StateGraph(MessagesState)
graph_builder.add_node("tool_calling_llm",tool_calling_llm)

graph_builder.add_edge(START,"tool_calling_llm")
graph_builder.add_edge("tool_calling_llm",END)

graph = graph_builder.compile()


In [27]:
response = graph.invoke({"messages":[HumanMessage(content="What is 3 * 12?")]})
for m in response['messages']:
    m.pretty_print()
    print("----------")

================================ Human Message =================================

What is 3 * 12?
----------
================================== Ai Message ==================================
Tool Calls:
  multiply (call_znt5)
 Call ID: call_znt5
  Args:
    a: 3
    b: 12
----------


In [28]:
response = graph.invoke({"messages":[HumanMessage(content="3*12 + 4")]})
for m in response['messages']:
    m.pretty_print()
    print("----------")

================================ Human Message =================================

3*12 + 4
----------
================================== Ai Message ==================================
Tool Calls:
  multiply (call_j1vh)
 Call ID: call_j1vh
  Args:
    a: 3
    b: 12
  add (call_0n4b)
 Call ID: call_0n4b
  Args:
    a: 36
    b: 4
----------
